In [16]:
import pandas as pd
import numpy as np
import os
import cv2
import PIL
from tempfile import TemporaryFile
from PIL import Image, ImageFile, Jpeg2KImagePlugin
import pickle
from pathlib import Path

import imageio
from pydicom import dcmread
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import matplotlib.pyplot as plt

In [9]:
train_df = pd.read_csv('train.csv')
train_img = ('train/')
ids = train_df.image_id

In [13]:


ids = train_df.image_id.unique()

df_data = []

for n, id_ in enumerate(ids):
    dicom_path = train_img + id_ + '.dicom'
    dicom = pydicom.dcmread(dicom_path)
    
    data = dicom.pixel_array
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    im = Image.fromarray(data)
    
    new_im = im.resize((256,256))
    npdata = np.asarray(new_im)
    
    npdata = npdata.flatten().reshape(1, 65536)
    df_data.append(npdata)


C:\Users\G-Sprouts\anaconda3\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
C:\Users\G-Sprouts\anaconda3\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


In [15]:
df_data

[array([[255, 255, 255, ...,   0,   0,   0]], dtype=uint8),
 array([[10,  9, 10, ...,  0,  0,  0]], dtype=uint8),
 array([[255, 255, 255, ..., 233, 238, 250]], dtype=uint8),
 array([[255, 255, 255, ...,  25,  26,  27]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[251, 250, 251, ...,  55,  49,  51]], dtype=uint8),
 array([[ 0,  0,  0, ..., 43, 38, 38]], dtype=uint8),
 array([[242, 242, 242, ..., 250, 250, 250]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 7, 7, 8]], dtype=uint8),
 array([[176, 178, 178, ..., 248, 248, 248]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[99, 99, 90, ..., 75, 76, 67]], dtype=uint8),
 array([[248, 248, 248, ..., 246, 247, 246]], dtype=uint8),
 array([[19, 18, 17, ...,  3,  3,  3]], dtype=uint8),
 array([[20, 21, 21, ..., 15, 12, 71]], dtype=uint8),
 array([[ 0,  0,  0, ...,  4,  6, 18]], dtype=uint8),
 array([[0, 0, 0

In [39]:
np.savez('arrays.npz', *df_data)

In [41]:
files = np.load('arrays.npz')

In [42]:
file = [files[key] for key in files]

In [43]:
file

[array([[255, 255, 255, ...,   0,   0,   0]], dtype=uint8),
 array([[10,  9, 10, ...,  0,  0,  0]], dtype=uint8),
 array([[255, 255, 255, ..., 233, 238, 250]], dtype=uint8),
 array([[255, 255, 255, ...,  25,  26,  27]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[251, 250, 251, ...,  55,  49,  51]], dtype=uint8),
 array([[ 0,  0,  0, ..., 43, 38, 38]], dtype=uint8),
 array([[242, 242, 242, ..., 250, 250, 250]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[0, 0, 0, ..., 7, 7, 8]], dtype=uint8),
 array([[176, 178, 178, ..., 248, 248, 248]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[99, 99, 90, ..., 75, 76, 67]], dtype=uint8),
 array([[248, 248, 248, ..., 246, 247, 246]], dtype=uint8),
 array([[19, 18, 17, ...,  3,  3,  3]], dtype=uint8),
 array([[20, 21, 21, ..., 15, 12, 71]], dtype=uint8),
 array([[ 0,  0,  0, ...,  4,  6, 18]], dtype=uint8),
 array([[0, 0, 0

In [ ]:
labels = []
for id_ in ids:
    df = train_data[train_data.image_id == id_]
    label = df.iloc[0,2]
    #label = tf.io.decode_raw(label, tf.uint8)
    #label = tf.reshape(label, label.shape)
    #label = tf.one_hot(label, 10)
    labels.append(label)